## numpy

In [1]:
import numpy as np

In [2]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.

N, D_in, H, D_out = 64, 1000, 100, 10

In [3]:
# 무작위의 입력과 출력 데이터를 생성합니다.
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

In [10]:
x.shape

(64, 1000)

In [4]:
# 무작위로 가중치를 초기화합니다.
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

In [14]:
print(
    w1.shape,
    w2.shape
)

(1000, 100) (100, 10)


In [17]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # 손실(loss)을 계산하고 출력합니다.
    loss = np.square(y_pred - y).sum()
    if t%100 == 0:
        print(t, loss)
    
    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred) #벡터미분
    grad_h_relu = grad_y_pred.dot(w2.T) #벡터미분
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h) #벡터미분
    
    # 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 1.0146681946648067e-08
100 3.453226572882835e-11
200 1.3123300121477219e-13
300 5.600151799602955e-16
400 2.653116518733606e-18


## PyTorch: Tensors

In [18]:
import torch

In [19]:
dtype = torch.float
device = torch.device('cuda:0')

In [27]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [28]:
# 무작위의 입력과 출력 데이터를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [29]:
#무작위로 가중치를 초기화합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

[선형대수 연산]
1. dot : 벡터의 내적
2. mv : 행렬과 벡터의 곱
3. mm : 행력과 행렬의 곱
4. matmul : 인수의 종류에 따라서 자동으로 dot, mv, mm을 선택

[기타 연산]
1. clamp(min, max) : input 되는 값의 상한/하한을 설정하면 그 기준으로 출력된다.

In [30]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.mm(w1) #행렬과 행렬의 곱
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # 손실(loss)을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    
    # 손실에 따른 w1, w2의 변화도를 계싼하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 692.4354248046875
199 3.5412392616271973
299 0.02804386243224144
399 0.0004838769091293216
499 6.411041249521077e-05


## PyTorch: Autograd

In [31]:
import torch

In [32]:
dtype = torch.float
device = torch.device('cuda:0')

In [33]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [34]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad = False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 계산할
# 필요가 없음을 나타냅니다.
# requires_grad의 기본값은False이다..
x = torch.randn(N, D_in, device=device, dtype=dtype, requires_grad=False)
y = torch.randn(N, D_out, device=device, dtype=dtype, requires_grad=False)

In [35]:
# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한
# 변화도를 계산할 필요가 있음을 나타냅니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [36]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y값을 계산합니다.
    # 이는 Tensor를 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로 구현하지 않아도 되므로
    # 중간값들에 대한 참조를 갖고 있을 필요가 없습니다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor 연산을 사용하여 손실을 계산하고 출력합니다.
    # loss는 (1, ) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값입니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산합니다.
    # 이는 requires_grad=True를 갖는 모든 Tensor에 대해 손실의 변화도를 계산합니다.
    # 이후 w1.grad와 w2.grad는 w1과 w2 각각에 대한 소실의 변화도를 갖는 Tensor가 됩니다.
    loss.backward()
    
    # 경사하강법을 사용하여 가중치를 수동으로 갱신합니다.
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True이지만
    # autograd에서는 이를 추적할 필요가 없기 때문입니다.
    # tensor.data가 tensor의 저장공간을 공유하기는 하지만, 이력을 추적하진 않습니다.
    # 또한, 이를 위해 torch.optim.SGD를 사용할 수도 있습니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        w1.grad.zero_()
        w2.grad.zero_()

99 1229.3853759765625
199 11.426041603088379
299 0.14126911759376526
399 0.002246161224320531
499 0.00016095700266305357


## PyTorch: 새로운 autograd 사용자 정의 함수 구현

In [37]:
import torch

In [98]:
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        """
        순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환합니다.
        ctx는 컨택스트 객체(context object)로 역전파 연산을 위한 정보 저장에 사용합니다.
        ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 어떠한 객체도 저장(cache)해 둘 수 있다.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 
        입력에 대한 손실의 변화도를 계산합니다.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

<b> @staticmethod VS. @classmethod</b> <br>
https://hamait.tistory.com/635

In [93]:
dtype = torch.float
device = torch.device('cuda:0')

In [94]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [95]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [96]:
# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad = True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad = True)

In [97]:
learning_rate = 1e-6
for t in range(500):
    # 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용한다.
    # 여기에 relu라는 이름을 붙여준다.
    relu = MyReLU.apply
    
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y값을 계산합니다.
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산합니다.
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 손실을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산합니다.
    loss.backward()
    
    # 경사하강법을 사용하여 가중치를 갱신합니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
    
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        w1.grad.zero_()
        w2.grad.zero_()
    

99 902.748046875
199 24.52218246459961
299 0.9584438800811768
399 0.04138381779193878
499 0.002255598548799753


## PyTorch: nn모듈

연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는 매우 강력한 패러다임입니다; 하지만 규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level)일 수 있습니다.

신경망을 구성할 때 종종 연산을 여러 계층 에 배열(arrange)하는 것으로 생각하는데, 이 중 일부는 학습 도중 최적화가 될 학습 가능한 매개변수 를 갖고 있습니다.

In [99]:
import torch

In [100]:
# N은 배치 크기이며 D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며 D_out은 출력의 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [101]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [102]:
# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의합니다.
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성합니다. 각각의 Linear Module은 선형 함수를 사용하여
# 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

In [103]:
# 또한 nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있습니다.
# 여기에서는 평균 제곱 오차(MSE)를 손실 함수로 사용하겠습니다.
loss_fn = torch.nn.MSELoss(reduction='sum')

In [105]:
learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    # Module 객체는 __call__ 연산자를 덮어써서(orverride) 함수처럼 호출할 수 있게 합니다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의 Tensor를 생성합니다.
    y_pred = model(x)
    
    # 손실을 계산하고 출력합니다.
    # 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실 값을 갖는 Tensor를 반환합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    # 역전파 단계를 실행하기 전에 변화도를 0으로 만듭니다.
    model.zero_grad()
    
    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를 계산합니다.
    # 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때 Tensor 내에 저장되므로,
    # 이 호출은 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산하게 됩니다.
    loss.backward()
    
    # 경사하강법을 사용하여 가중치를 갱신합니다.
    # 각 매개변수는 Tensor이므로 이전에 했던 것과 같이 변화도에 접근할 수 있습니다.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.9997453689575195
199 0.08178296685218811
299 0.0035355333238840103
399 0.00017878200742416084
499 9.910208973451518e-06


## PyTorch: optim

In [106]:
import torch

In [107]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력의 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [108]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [110]:
# nn패키지를 사용하여 모델과 손실함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
loss_fn = torch.nn.MSELoss(reduction = 'sum')

In [114]:
# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의합니다.
# 여기서는 Adam을 사용하겠습니다.
# optim 패키지는 다른 다양한 최적화 알고리즘을 포함하고 있습니다.
# Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지 알려줍니다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y값을 계산합니다.
    y_pred = model(x)
    
    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만듭니다.
    # 이렇게 하는 이유는 기본적으로 backward()를 호출할 때마다 변화도가 버퍼에 누적되기 때문입니다.
    # 더 자세한 내용은 torch.autograd.backward에 대한 문서를 참조하세요. (아래 링크 첨부)
    optimizer.zero_grad()
    
    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산합니다.
    loss.backward()
    
    # Optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()

99 64.71513366699219
199 1.0224515199661255
299 0.009080538526177406
399 5.156657789484598e-05
499 1.1252154763496947e-07


## 사용자 정의 nn.Module

In [115]:
import torch

In [116]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 2개의 nn.Linear 모듈을 생성하고, 멤버 변수로 지정합니다.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 Tensor를 받고 출력 데이터의 Tensor를 반환해야 합니다.
        Tensor 상의 임의의 연산자뿐만 아니라 생성자에서 정의한 Module도 사용할 수 있습니다.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [117]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [118]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [119]:
# 앞에서 정의한 클래스를 생성하여 모델을 구성합니다.
model = TwoLayerNet(D_in, H, D_out)

In [120]:
# 손실 함수와 Optimizer를 만듭니다.
# SGD 생성자에 model.parameters()를 호출하면 모델의 멤버인 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함됩니다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산합니다.
    y_pred = model(x)
    
    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.6860620975494385
199 0.10616956651210785
299 0.009868842549622059
399 0.001078904839232564
499 0.000122667130199261
